<a href="https://colab.research.google.com/github/Josh-techie/AI-TPs/blob/master/Web_Scraping/Workshop_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b>Atelier II : Data Collection </b></h1></center>
<center><h3> Data Science</h3></center>
<center><h3>14 Mars 2024</h3></center>

---



# Exercise I:
https://www.annuaire-gratuit.ma/ is a directory that contains over 189,000 addresses, prayer times, emergency pharmacies, train schedules, prayer times...
In this exercise, we focus on emergency pharmacies in Moroccan cities.

- Create a bot that collects emergency pharmacies in the cities:
Agadir, Casablanca, Rabat, Fes, Tangier, Safi, Kenitra, Marrakech, Taza, Ouarzazate, Tiznit, Nador.
- Store this information in a DataFrame (pandas).
- Save the DataFrames in a csv file.




**To use secret Key in the notebook, use the following**

In [ ]:
from google.colab import userdata
userdata.get('OPENWEATHERAPI')

In [2]:
user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
cities = ['Agadir', 'Casablanca', 'Rabat', 'Fes', 'Tanger', 'Safi', 'Kenitra', 'Marrakech', 'Taza', 'Ouarzazate', 'Tiznit', 'Nador']

In [3]:
!pip install beautifulsoup4

In [4]:
import pandas as pd

In [5]:
import bs4 as BeautifulSoup

In [6]:
import requests

In [7]:
from bs4 import BeautifulSoup

In [8]:
# cities we want to scrape:
cities = ["Agadir", "Casablanca", "Rabat", "Fes"]

# first start by creating a list of the attributes we wants

pharmacy_names = []
pharmacy_addresses = []
pharmacy_telephones = []

# then we loop through the cities and scrape the data

for city in cities:
    page_to_scrape = f"https://www.annuaire-gratuit.ma/pharmacie-garde-{city}.html"

    # send the request to that particular url
    response = requests.get(page_to_scrape)
    # print(page_to_scrape)
    # check the status code and handle errors
    if response.status_code == 200:
        print("Request was successful")
        # specify the parser html.parser
        soup = BeautifulSoup(response.text, "html.parser")

        # parse pharmacy names then append to the list
        pharmcy_name = soup.find_all("h3", itemprop="name")
        for ph_name in pharmcy_name:
            # print(ph_name.text)
            pharmacy_names.append(ph_name.text)

        # parse pharmacy addresses then append to the list
        pharmacy_addres = soup.find_all("p", itemprop="streetAddress")
        for ph_add in pharmacy_addres:
            # print(ph_add.text)
            pharmacy_addresses.append(ph_add.text)

        phoneNumbers = soup.find_all(style="font-weight: bold; color:#1e6dab")
        # print(phoneNumbers.text)

        # parse pharmacy phone numbers then append to the list
        for phoneNumber in phoneNumbers:
            # print(phoneNumber.text)
            pharmacy_telephones.append(phoneNumber.text)
            # print(pharmacy_telephones)

        # here we'll create the dataframe
        data_pharmacies = pd.DataFrame(
            {
                "Name": pharmacy_names,
                "Address": pharmacy_addresses,
                "Telephone": pharmacy_telephones,
            }
        )
        # pharmacy_data = pd.DataFrame(city)

        # here I'll export the  data as csv
        data_pharmacies.to_csv(f"{city}.csv", index=False)
        # print(response.text)
        pharmacy_names.__len__()
        # clear the lists
        pharmacy_names.clear()
        pharmacy_addresses.clear()
        pharmacy_telephones.clear()
    else:
        print("Request Failed, with status code", response.status_code)

Request was successful
Request was successful
Request was successful
Request was successful


**Print the dataframe so later I'll merge both don't worry you'll understand once you read the second exercice 🌟** erf fuck I'll have to scrape the city as well later 😥

In [37]:
print(data_pharmacies)

                  Name                                            Address  \
0     Pharmacie Maalal  Pharmacie Maalal, n° 64 lot. hiba n°1 oued fes...   
1      Pharmacie Driss  Pharmacie Driss, 2 lot. almssala, hay al mssal...   
2  Pharmacie Ibn Nafis  Pharmacie Ibn Nafis, hay ain chkef dommaine ai...   
3   Pharmacie Al Aziza  Pharmacie Al Aziza, numéro 1 lotissement al az...   
4  Pharmacie El Otmani  Pharmacie El Otmani, 52 bd boufekran aouinate ...   
5   Pharmacie Al Ahbab  Pharmacie Al Ahbab, 254 lot al fadila 3 route ...   

   Telephone  
0  053564...  
1  053200...  
2  061441...  
3  053560...  
4  053596...  
5  053487...  


# **Exercise II:**

OpenWeatherMap is an online service, owned by OpenWeather Ltd, that provides global weather data through an API, including current weather data, forecasts, immediate forecasts, and historical weather data for any geographical location.

* Create a bot that collects the minimum temperature, maximum temperature, humidity, longitude, and latitude of the cities from Exercise I.
* Store this information in a DataFrame (pandas).
* Merge the DataFrame with the one you created in Exercise 1.
* Save the DataFrames to a csv file.

**You must first register to obtain a key in order to send requests to this API.**



**We will create a function to convert from Fahrenheit to Celsius since, we get data in Fahrenheit from the API**

In [42]:
# function to convert from Fahrenheit to Celsius
def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5/9
    return celsius

In [43]:
import json

temp_min = []
temp_max = []
humidity = []

from google.colab import userdata

# Retrieve the OpenWeatherMap API key from userdata
OPENWEATHERAPI = userdata.get('OPENWEATHERAPI')

# Ensure OPENWEATHERAPI is not None
if OPENWEATHERAPI is None:
    print("OpenWeatherMap API key not found. Please store your API key in userdata.")
else:
  # create a  list where I will store all the data
  cities_weather_data = []
  for city in cities:
    page = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHERAPI}")
    d = json.loads(page.content)

    # Extract min and max temperature and humidity from the JSON response
    min_temp = fahrenheit_to_celsius(d["main"]["temp_min"])
    max_temp = fahrenheit_to_celsius(d["main"]["temp_max"])
    humidity = d["main"]["humidity"]


    # append each scraped data for each iteration and append it to the list of dic
    cities_weather_data.append({
        "City": city,
        "Min Temperature (C⁰)": min_temp,
        "Max Temperature (C⁰)": max_temp,
        "Humidity": humidity,
    })

    # Print the weather data for the city
    print(f"The weather in {city} is: Min Temp: {min_temp}, Max Temp: {max_temp}, Humidity: {humidity} ")

The weather in Agadir is: Min Temp: 143.55555555555554, Max Temp: 143.55555555555554, Humidity: 85 
The weather in Casablanca is: Min Temp: 142.86666666666667, Max Temp: 144.5666666666667, Humidity: 59 
The weather in Rabat is: Min Temp: 140.10555555555555, Max Temp: 142.6277777777778, Humidity: 93 
The weather in Fes is: Min Temp: 142.38333333333333, Max Temp: 142.38333333333333, Humidity: 67 


**Store this information in a DataFrame (pandas)**

> Create a DataFrame from the list of dictionaries ( Cities_weather_data )





In [44]:
# Create a DataFrame from the list of dictionaries
cities_weather_df = pd.DataFrame(cities_weather_data)
print(cities_weather_df)

         City  Min Temperature (C⁰)  Max Temperature (C⁰)  Humidity
0      Agadir            143.555556            143.555556        85
1  Casablanca            142.866667            144.566667        59
2       Rabat            140.105556            142.627778        93
3         Fes            142.383333            142.383333        67


**Let's now read the csv we created previouslty for each city ◀**




> For Agadir 💪






In [46]:
agadir = pd.read_csv('Agadir.csv')

agadir_merged = pd.merge(agadir.assign(key=1), cities_weather_df[cities_weather_df['City'] == 'Agadir'].assign(key=1), on='key', how='left').drop('key', axis=1)

agadir_merged.to_csv('Agadir_final.csv', index=False)

**Same for the other cities 🎈**
> Otherwise use a for loop

In [47]:
Casablanca = pd.read_csv('Casablanca.csv')

Casablanca_merged = pd.merge(agadir.assign(key=1), cities_weather_df[cities_weather_df['City'] == 'Casablanca'].assign(key=1), on='key', how='left').drop('key', axis=1)

Casablanca_merged.to_csv('Casablanca_final.csv', index=False)

# fes

